In [ ]:
"""
╔══════════════════════════════════════════════════════════════════════════════╗
║                                                                              ║
║              🤖 SMOLAGENTS WORKSHOP: BUILD YOUR FIRST AI AGENT 🤖           ║
║                                                                              ║
║  Welcome! In this hands-on workshop, you'll learn how to build AI agents     ║
║  that can use tools to accomplish complex tasks.                             ║
║                                                                              ║
║  📚 What you'll learn:                                                      ║
║     ✓ What agents and tools are                                             ║
║     ✓ How to create custom tools                                            ║
║     ✓ How to build agents that use your tools                               ║
║     ✓ Best practices for production-ready agents                            ║
║                                                                              ║
║  ⏱️ Estimated time: 90 minutes                                              ║
║  🎯 Difficulty: Beginner to Intermediate                                    ║
║                                                                              ║
╚══════════════════════════════════════════════════════════════════════════════╝
"""

# ============================================================================
# 📦 SECTION 1: INSTALLATION & SETUP (5 minutes)
# ============================================================================

print("🚀 Starting Section 1: Installation & Setup\n")

"""
WHAT ARE AI AGENTS?
-------------------
An AI Agent is a system that:
1. 🧠 THINKS: Uses an LLM to understand tasks and plan actions
2. 🔧 ACTS: Executes actions using "tools" (functions it can call)
3. 👀 OBSERVES: Sees the results of its actions
4. 🔄 ITERATES: Repeats the cycle until the task is complete

WHAT ARE TOOLS?
---------------
Tools are functions that give your agent capabilities. For example:
- A web search tool lets it find information online
- A calculator tool lets it perform mathematical operations
- A weather tool lets it check the weather
- A custom tool lets it do ANYTHING you can code!

WHY SMOLAGENTS?
---------------
smolagents is a lightweight library that makes building agents simple:
✓ Minimal code complexity
✓ Easy to understand and extend
✓ Code-first approach (agents write Python, not JSON)
✓ Works with any LLM via Hugging Face
"""

# Install required packages
# Uncomment the line below if running in Google Colab or a fresh environment
# !pip install smolagents huggingface_hub -U -q

print("✅ Installing smolagents...")
print("✅ Installing huggingface_hub...")

# Import necessary libraries
from smolagents import (
    CodeAgent,           # The main agent that writes Python code
    tool,                # Decorator to create simple tools
    Tool,                # Class to create advanced tools
    InferenceClientModel # To connect to Hugging Face models
)
from huggingface_hub import login
import os

print("\n📚 Libraries imported successfully!")

# ============================================================================
# 🔐 AUTHENTICATE WITH HUGGING FACE
# ============================================================================

print("\n" + "="*80)
print("🔐 HUGGING FACE AUTHENTICATION")
print("="*80)

"""
WHY DO WE NEED TO AUTHENTICATE?
--------------------------------
We need a Hugging Face token to access the Serverless Inference API,
which lets us use powerful LLMs without running them locally.

HOW TO GET YOUR TOKEN:
1. Go to https://huggingface.co/settings/tokens
2. Create a new token with "Inference" permission
3. Copy the token
4. Paste it below when prompted
"""

# Authenticate with Hugging Face
# OPTION 1: Login interactively (recommended for notebooks)
print("\n📝 Please enter your Hugging Face token when prompted...")
login()

# OPTION 2: Use environment variable (recommended for scripts)
# os.environ["HF_TOKEN"] = "your_token_here"

print("✅ Authentication successful!\n")

In [ ]:
# ============================================================================
# 🎯 SECTION 2: OBSERVE A WORKING AGENT (10 minutes)
# ============================================================================

print("="*80)
print("🎯 SECTION 2: YOUR FIRST AGENT - A SIMPLE CALCULATOR")
print("="*80)

"""
Let's start by observing a COMPLETE, WORKING agent.
We'll build an agent with a simple calculator tool, then analyze how it works.

LEARNING OBJECTIVES:
- Understand tool structure
- See how agents use tools
- Observe the agent's reasoning process
"""

# ----------------------------------------------------------------------------
# STEP 1: CREATE A TOOL
# ----------------------------------------------------------------------------

@tool
def calculator(operation: str, num1: float, num2: float) -> float:
    """
    Performs basic mathematical operations.
    
    This is a TOOL DEFINITION. Let's break down each component:
    
    1️⃣ @tool DECORATOR:
       - This tells smolagents "this function is a tool the agent can use"
       - It automatically processes the function signature and docstring
    
    2️⃣ FUNCTION NAME (calculator):
       - Should be descriptive and clear
       - The LLM will see this name and decide when to use it
       - Use snake_case (lowercase with underscores)
    
    3️⃣ TYPE HINTS (operation: str, num1: float, num2: float) -> float:
       - CRITICAL! The agent needs to know what types to pass
       - operation: str = the input must be a string
       - num1: float = the input must be a number
       - -> float = the function returns a number
       - Always include type hints for inputs AND outputs!
    
    4️⃣ DOCSTRING (the text in triple quotes):
       - The MOST IMPORTANT part for the LLM
       - This is how the agent understands WHAT the tool does
       - Must include an Args: section describing each parameter
       - Be clear and specific!
    
    Args:
        operation: The operation to perform. Must be one of: 'add', 'subtract', 'multiply', 'divide'
        num1: The first number
        num2: The second number
    
    Returns:
        The result of the mathematical operation
    
    Example:
        calculator('add', 5, 3) returns 8.0
    """
    
    # The actual logic of our tool - simple Python code!
    if operation == "add":
        return num1 + num2
    elif operation == "subtract":
        return num1 - num2
    elif operation == "multiply":
        return num1 * num2
    elif operation == "divide":
        if num2 == 0:
            return "Error: Division by zero!"
        return num1 / num2
    else:
        return f"Error: Unknown operation '{operation}'"


print("✅ Calculator tool created!\n")

# Let's test our tool directly (before giving it to an agent)
print("🧪 Testing the tool directly:")
print(f"   5 + 3 = {calculator('add', 5, 3)}")
print(f"   10 - 4 = {calculator('subtract', 10, 4)}")
print(f"   6 * 7 = {calculator('multiply', 6, 7)}")
print(f"   20 / 4 = {calculator('divide', 20, 4)}\n")

In [ ]:
# ----------------------------------------------------------------------------
# STEP 2: CREATE AN AGENT
# ----------------------------------------------------------------------------

print("🤖 Creating an agent with the calculator tool...\n")

"""
Now let's create an agent that can use our calculator tool.

THE AGENT CREATION PROCESS:
"""

# Initialize the language model
# This is the "brain" of our agent
model = InferenceClientModel(
    model_id="Qwen/Qwen2.5-Coder-32B-Instruct",  # A powerful, free model good at code
    # You can use other models like:
    # - "meta-llama/Llama-3.1-70B-Instruct" (very powerful)
    # - "mistralai/Mistral-7B-Instruct-v0.3" (smaller, faster)
)

print("✅ Language model initialized: Qwen/Qwen2.5-Coder-32B-Instruct")

# Create the agent
agent = CodeAgent(
    tools=[calculator],  # 🔧 LIST OF TOOLS: Give the agent access to our calculator
                        #    You can add multiple tools here: [tool1, tool2, tool3]
    
    model=model,        # 🧠 THE LLM BRAIN: The language model that powers the agent
    
    max_steps=6,        # 🔄 SAFETY LIMIT: Maximum number of think-act-observe cycles
                        #    Prevents infinite loops
    
    verbosity_level=2   # 📊 DEBUG OUTPUT: 
                        #    0 = silent, 1 = minimal, 2 = detailed (recommended for learning)
)

print("✅ Agent created and ready!\n")

In [ ]:
# ----------------------------------------------------------------------------
# STEP 3: RUN THE AGENT
# ----------------------------------------------------------------------------

print("="*80)
print("🚀 RUNNING THE AGENT")
print("="*80)
print("\nTask: Calculate (15 + 25) * 2 - 10\n")

"""
WHAT WILL HAPPEN:
1. The agent receives the task
2. It "thinks" about how to solve it (break it into steps)
3. It "acts" by calling the calculator tool
4. It "observes" the results
5. It repeats until done

Watch the output below to see the agent's reasoning!
"""

result = agent.run(
    "What is (15 + 25) multiplied by 2, minus 10? Use the calculator tool to solve this step by step."
)

print("\n" + "="*80)
print(f"✨ FINAL ANSWER: {result}")
print("="*80)

"""
📊 ANALYZING WHAT JUST HAPPENED:

Look at the output above. You should see:
1. The agent's THOUGHTS about how to approach the problem
2. The PYTHON CODE it wrote to call the calculator tool
3. The OBSERVATIONS from executing the code
4. The reasoning process repeating until it reached a final answer

This is the THINK → ACT → OBSERVE → REPEAT cycle in action!
"""

In [ ]:
"""
⚠️ WARNING: This is an ANTI-PATTERN example!
This demonstrates how BAD naming and documentation can break agent reasoning.

LEARNING OBJECTIVES:
- Understand why clear tool documentation is CRITICAL
- See how confusing names mislead agents
- Learn what NOT to do when creating tools
"""

# ----------------------------------------------------------------------------
# STEP 1: CREATE A CONFUSING TOOL (DON'T DO THIS!)
# ----------------------------------------------------------------------------

@tool
def text_string_concatenator_pro(mode: str, value_a: float, value_b: float) -> float:
    """
    Advanced string manipulation and text processing utility.
    
    ⚠️ DELIBERATELY CONFUSING ELEMENTS:
    
    1️⃣ MISLEADING NAME:
       - Name says "text_string_concatenator" but it does MATH
       - The agent will think this tool is for text/strings
       - The "pro" suffix is meaningless fluff
    
    2️⃣ WRONG DESCRIPTION:
       - Says "string manipulation" but does calculations
       - Completely misleading!
    
    3️⃣ VAGUE PARAMETER NAMES:
       - "mode" instead of "operation" (less clear)
       - "value_a" and "value_b" instead of "num1" and "num2"
       - Not obviously mathematical
    
    4️⃣ CONTRADICTORY DOCSTRING:
       - Says it processes text, but parameters are floats
       - Type hints say float, description says string
       - Agent will be confused about what this does!
    
    This tool concatenates text strings together in various ways.
    Use this when you need to join, merge, or combine textual data.
    Perfect for string operations and text manipulation tasks.
    
    Args:
        mode: The concatenation strategy (options: 'join', 'remove', 'repeat', 'split')
        value_a: The first text segment to process
        value_b: The second text segment to process
    
    Returns:
        The concatenated or processed string result
    
    Note: 
        - This is optimized for large text documents
        - Supports Unicode and special characters
        - Best used for formatting reports and documents
    """
    
    # The ACTUAL logic - it's just a calculator! 🤦
    if mode == "add":
        return value_a + value_b
    elif mode == "subtract":
        return value_a - value_b
    elif mode == "multiply":
        return value_a * value_b
    elif mode == "divide":
        if value_b == 0:
            return "Error: Division by zero!"
        return value_a / value_b
    else:
        return f"Error: Unknown operation '{mode}'"


print("⚠️ Confusing tool created! (This will break agent reasoning)\n")


In [ ]:
# ----------------------------------------------------------------------------
# STEP 2: CREATE AN AGENT
# ----------------------------------------------------------------------------

print("🤖 Creating an agent with the text_string_concatenator_pro tool...\n")

"""
Now let's create an agent that can use our this confusing calculator tool.

THE AGENT CREATION PROCESS:
"""

# Initialize the language model
# This is the "brain" of our agent
model = InferenceClientModel(
    model_id="Qwen/Qwen2.5-Coder-32B-Instruct",  # A powerful, free model good at code
    # You can use other models like:
    # - "meta-llama/Llama-3.1-70B-Instruct" (very powerful)
    # - "mistralai/Mistral-7B-Instruct-v0.3" (smaller, faster)
)

print("✅ Language model initialized: Qwen/Qwen2.5-Coder-32B-Instruct")

# Create the agent
agent = CodeAgent(
    tools=[text_string_concatenator_pro],  # 🔧 LIST OF TOOLS: Give the agent access to our calculator
                        #    You can add multiple tools here: [tool1, tool2, tool3]
    
    model=model,        # 🧠 THE LLM BRAIN: The language model that powers the agent
    
    max_steps=6,        # 🔄 SAFETY LIMIT: Maximum number of think-act-observe cycles
                        #    Prevents infinite loops
    
    verbosity_level=2   # 📊 DEBUG OUTPUT: 
                        #    0 = silent, 1 = minimal, 2 = detailed (recommended for learning)
)

print("✅ Agent created and ready!\n")

In [ ]:
# ----------------------------------------------------------------------------
# STEP 3: RUN THE AGENT
# ----------------------------------------------------------------------------

print("="*80)
print("🚀 RUNNING THE AGENT")
print("="*80)
print("\nTask: Calculate (15 + 25) * 2 - 10\n")

"""
WHAT WILL HAPPEN:
1. The agent receives the task
2. It "thinks" about how to solve it (break it into steps)
3. It "acts" by calling the text_string_concatenator_pro tool
4. It "observes" the results
5. It repeats until done

Watch the output below to see the agent's reasoning!
"""

result = agent.run(
    "What is (15 + 25) multiplied by 2, minus 10? Use the text_string_concatenator_pro tool to solve this step by step."
)

print("\n" + "="*80)
print(f"✨ FINAL ANSWER: {result}")
print("="*80)


In [ ]:


# ----------------------------------------------------------------------------
# STEP 1: CREATE A bad agent TOOL
# ----------------------------------------------------------------------------

@tool
def bad_calculator(operation: str, num1: float, num2: float) -> float:
    """
    Performs basic mathematical operations.
    
    This is a TOOL DEFINITION. Let's break down each component:
    
    1️⃣ @tool DECORATOR:
       - This tells smolagents "this function is a tool the agent can use"
       - It automatically processes the function signature and docstring
    
    2️⃣ FUNCTION NAME (bad_calculator):
       - Should be descriptive and clear
       - The LLM will see this name and decide when to use it
       - Use snake_case (lowercase with underscores)
    
    3️⃣ TYPE HINTS (operation: str, num1: float, num2: float) -> float:
       - CRITICAL! The agent needs to know what types to pass
       - operation: str = the input must be a string
       - num1: float = the input must be a number
       - -> float = the function returns a number
       - Always include type hints for inputs AND outputs!
    
    4️⃣ DOCSTRING (the text in triple quotes):
       - The MOST IMPORTANT part for the LLM
       - This is how the agent understands WHAT the tool does
       - Must include an Args: section describing each parameter
       - Be clear and specific!
    
    Args:
        operation: The operation to perform. Must be one of: 'add', 'subtract', 'multiply', 'divide'
        num1: The first number
        num2: The second number
    
    Returns:
        The bad result of the mathematical operation
    
    Example:
        bad_calculator('add', 5, 3) returns 15
    """
    
    # The actual logic of our tool - simple Python code!
    if operation == "add":
        return num1 * num2
    elif operation == "subtract":
        return num1 + num2
    elif operation == "multiply":
        return num1 / num2
    elif operation == "divide":
        if num2 == 0:
            return "Error: Division by zero!"
        return num1 / num2
    else:
        return f"Error: Unknown operation '{operation}'"


print("✅ Calculator tool created!\n")

# Let's test our tool directly (before giving it to an agent)
print("🧪 Testing the tool directly:")
print(f"   5 + 3 = {bad_calculator('add', 5, 3)}")
print(f"   10 - 4 = {bad_calculator('subtract', 10, 4)}")
print(f"   6 * 7 = {bad_calculator('multiply', 6, 7)}")
print(f"   20 / 4 = {bad_calculator('divide', 20, 4)}\n")

In [ ]:
# ----------------------------------------------------------------------------
# STEP 2: CREATE AN AGENT
# ----------------------------------------------------------------------------

print("🤖 Creating an agent with the bad_calculator tool...\n")

"""
Now let's create an agent that can use our bad_calculator tool.

THE AGENT CREATION PROCESS:
"""

# Initialize the language model
# This is the "brain" of our agent
model = InferenceClientModel(
    model_id="Qwen/Qwen2.5-Coder-32B-Instruct",  # A powerful, free model good at code
    # You can use other models like:
    # - "meta-llama/Llama-3.1-70B-Instruct" (very powerful)
    # - "mistralai/Mistral-7B-Instruct-v0.3" (smaller, faster)
)

print("✅ Language model initialized: Qwen/Qwen2.5-Coder-32B-Instruct")

# Create the agent
agent = CodeAgent(
    tools=[bad_calculator],  # 🔧 LIST OF TOOLS: Give the agent access to our calculator
                        #    You can add multiple tools here: [tool1, tool2, tool3]
    
    model=model,        # 🧠 THE LLM BRAIN: The language model that powers the agent
    
    max_steps=6,        # 🔄 SAFETY LIMIT: Maximum number of think-act-observe cycles
                        #    Prevents infinite loops
    
    verbosity_level=2   # 📊 DEBUG OUTPUT: 
                        #    0 = silent, 1 = minimal, 2 = detailed (recommended for learning)
)

print("✅ Agent created and ready!\n")

In [ ]:
# ----------------------------------------------------------------------------
# STEP 3: RUN THE AGENT
# ----------------------------------------------------------------------------

print("="*80)
print("🚀 RUNNING THE AGENT")
print("="*80)
print("\nTask: Calculate (15 + 25) * 2 - 10\n")

"""
WHAT WILL HAPPEN:
1. The agent receives the task
2. It "thinks" about how to solve it (break it into steps)
3. It "acts" by calling the bad_calculator tool
4. It "observes" the results
5. It repeats until done

Watch the output below to see the agent's reasoning!
"""

result = agent.run(
    "What is (15 + 25) multiplied by 2, minus 10? Use the bad_calculator tool to solve this step by step."
)

print("\n" + "="*80)
print(f"✨ FINAL ANSWER: {result}")
print("="*80)

"""
📊 ANALYZING WHAT JUST HAPPENED:

Look at the output above. You should see:
1. The agent's THOUGHTS about how to approach the problem
2. The PYTHON CODE it wrote to call the bad_calculator tool
3. The OBSERVATIONS from executing the code
4. The reasoning process repeating until it reached a final answer

This is the THINK → ACT → OBSERVE → REPEAT cycle in action!
"""

print("\n✅ Section 2 complete! You've seen a working agent in action.\n")

In [ ]:
# ============================================================================
# 📚 SECTION 3: UNDERSTANDING TOOL ANATOMY (10 minutes)
# ============================================================================

print("="*80)
print("📚 SECTION 3: DEEP DIVE INTO TOOL STRUCTURE")
print("="*80)

"""
Now that you've seen a tool in action, let's understand what makes a GOOD tool.

THE ANATOMY OF A PERFECT TOOL:
"""

# ----------------------------------------------------------------------------
# EXAMPLE 1: A SIMPLE TOOL USING @tool DECORATOR
# ----------------------------------------------------------------------------

print("\n1️⃣ METHOD 1: Using the @tool decorator (RECOMMENDED FOR SIMPLE TOOLS)\n")

@tool
def word_counter(text: str) -> int:
    """
    Counts the number of words in a given text.
    
    ✅ GOOD PRACTICES DEMONSTRATED HERE:
    - Clear, descriptive function name
    - Type hints for input (str) and output (int)
    - Detailed docstring explaining what it does
    - Args section describing the parameter
    
    Args:
        text: The text to count words in
    
    Returns:
        The number of words in the text
    """
    return len(text.split())

print("✅ Simple tool created with @tool decorator")
print(f"   Example: word_counter('Hello world from AI') = {word_counter('Hello world from AI')}\n")

In [ ]:
# ----------------------------------------------------------------------------
# EXAMPLE 2: AN ADVANCED TOOL USING Tool CLASS  TODO
# ----------------------------------------------------------------------------

print("2️⃣ METHOD 2: Using the Tool class (FOR COMPLEX TOOLS)\n")

class TemperatureConverter(Tool):
    """
    When you need more control, use the Tool class.
    This is useful for:
    - Complex initialization
    - Maintaining state
    - Advanced error handling
    """
    
    # 1️⃣ NAME: How the agent will call this tool
    name = "temperature_converter"
    
    # 2️⃣ DESCRIPTION: What the tool does (shown to the LLM)
    description = "Converts temperatures between Celsius and Fahrenheit."
    
    # 3️⃣ INPUTS: Define the structure of input parameters
    inputs = {
        "value": {
            "type": "number",  # Can be: "string", "number", "boolean", "array", etc.
            "description": "The temperature value to convert"
        },
        "from_unit": {
            "type": "string",
            "description": "The source unit: 'C' for Celsius or 'F' for Fahrenheit"
        },
        "to_unit": {
            "type": "string",
            "description": "The target unit: 'C' for Celsius or 'F' for Fahrenheit"
        }
    }
    
    # 4️⃣ OUTPUT TYPE: What the tool returns
    output_type = "number"
    
    # 5️⃣ FORWARD METHOD: The actual logic (like __call__ or run)
    def forward(self, value: float, from_unit: str, to_unit: str) -> float:
        """
        This method contains the actual tool logic.
        It's called automatically when the agent uses the tool.
        """
        # Convert to uppercase for consistency
        from_unit = from_unit.upper()
        to_unit = to_unit.upper()
        
        # Validation
        if from_unit not in ['C', 'F'] or to_unit not in ['C', 'F']:
            raise ValueError("Units must be 'C' or 'F'")
        
        # Same unit, no conversion needed
        if from_unit == to_unit:
            return value
        
        # Celsius to Fahrenheit
        if from_unit == 'C' and to_unit == 'F':
            return (value * 9/5) + 32
        
        # Fahrenheit to Celsius
        if from_unit == 'F' and to_unit == 'C':
            return (value - 32) * 5/9
        
        return value

# Instantiate the tool (IMPORTANT: You must create an instance!)
temp_converter = TemperatureConverter()

print("✅ Advanced tool created with Tool class")
print(f"   Example: 0°C to Fahrenheit = {temp_converter.forward(0, 'C', 'F')}°F")
print(f"   Example: 98.6°F to Celsius = {temp_converter.forward(98.6, 'F', 'C'):.1f}°C\n")

In [ ]:
# === SECTION 3 — AI ROUTER DEMO ==============================================
# 🎯 Objectif : laisser le modèle choisir automatiquement entre
# word_counter et temperature_converter selon la consigne utilisateur

# 1️⃣ Initialisation du modèle (même que dans la section 2)
model = InferenceClientModel(
    model_id="Qwen/Qwen2.5-Coder-32B-Instruct",  # excellent modèle gratuit pour le code
)
print("✅ Language model initialized: Qwen/Qwen2.5-Coder-32B-Instruct")

# 2️⃣ Création d’un agent ayant accès à nos deux tools
agent = CodeAgent(
    tools=[word_counter, temp_converter],  # outils disponibles
    model=model,                           # cerveau LLM
    max_steps=6,                           # évite les boucles infinies
    verbosity_level=2                      # affichage détaillé pour comprendre le raisonnement
)

# 3️⃣ Prompts de test (6 exemples)
prompts = [
    "How many words are in: 'Knowledge is power but enthusiasm pulls the switch'?",
    "Convert 21 C to F.",
    "Convert 70 F to C, please.",
    "Count the words in: 'Machine learning is the new electricity.'",
    "Convert -10 C to F.",
    "How many words are in: 'tiny models, big impact'?"
]

print("\n" + "="*80)
print("🤖 AI ROUTER — Let the model decide which tool to use")
print("="*80)

# 4️⃣ Exécution des requêtes
for p in prompts:
    print("\n— User:", p)
    out = agent.run(p)
    print("→ Agent:", out)


In [ ]:
# ============================================================================
# 🚀 SECTION 4: BUILD YOUR OWN TOOL FROM SCRATCH (30 minutes)
# ============================================================================

print("="*80)
print("🚀 SECTION 4: CREATE YOUR OWN TOOL FROM SCRATCH")
print("="*80)
# ============================================================================
# 🔴 DATA ANALYSIS TOOL
# ============================================================================

print("\n" + "="*80)
print("🔴  BUILD A DATA ANALYSIS TOOL")
print("="*80)

"""
GOAL: Create a tool that performs statistical analysis on numbers

REQUIREMENTS:
✓ Function name: data_analyzer
✓ Inputs: numbers (list of floats), analysis_type (str)
✓ Analysis types to support:
  - 'mean': Calculate average
  - 'median': Find middle value
  - 'mode': Find most common value
  - 'range': Find difference between max and min
  - 'std': Calculate standard deviation
✓ Return: The analysis result (float or str)
✓ Handle edge cases (empty list, etc.)
"""

# TODO: Complete this tool using the Tool class!

class DataAnalyzer(Tool):
    """
    # TODO: Write a class docstring
    """
    
    # TODO: Define the tool name
    name = ""  # TODO
    
    # TODO: Write a description
    description = ""  # TODO
    
    # TODO: Define inputs
    inputs = {
        # TODO: Add 'numbers' input
        # TODO: Add 'analysis_type' input
    }
    
    # TODO: Define output type
    output_type = ""  # TODO
    
    def forward(self, numbers: list, analysis_type: str):
        """
        # TODO: Document this method
        """
        
        # TODO: Handle empty list
        if not numbers:
            return "Error: Cannot analyze empty list"
        
        # TODO: Implement each analysis type
        
        if analysis_type == "mean":
            # TODO: Calculate and return mean (average)
            # HINT: sum(numbers) / len(numbers)
            pass
            
        elif analysis_type == "median":
            # TODO: Calculate and return median
            # HINT: Sort the list first, then find middle value
            pass
            
        elif analysis_type == "mode":
            # TODO: Find and return most common value
            # HINT: Use max() with key parameter
            pass
            
        elif analysis_type == "range":
            # TODO: Calculate and return range
            # HINT: max(numbers) - min(numbers)
            pass
            
        elif analysis_type == "std":
            # TODO: Calculate standard deviation
            # HINT: sqrt(sum((x - mean)^2) / n)
            pass
            
        else:
            return f"Error: Unknown analysis type '{analysis_type}'"

# TODO: Create an instance of your tool
# data_analyzer = DataAnalyzer()

In [ ]:
# SOLUTION (Try on your own first!)

class DataAnalyzer(Tool):
    '''
    Performs statistical analysis on numerical data.
    '''
    
    name = "data_analyzer"
    description = "Analyzes a list of numbers and returns statistical information"
    
    inputs = {
        "numbers": {
            "type": "array",
            "description": "A list of numbers to analyze"
        },
        "analysis_type": {
            "type": "string",
            "description": "Type of analysis: 'mean', 'median', 'mode', 'range', or 'std'"
        }
    }
    
    output_type = "number"
    
    def forward(self, numbers: list, analysis_type: str):
        if not numbers:
            return "Error: Cannot analyze empty list"
        
        if analysis_type == "mean":
            return sum(numbers) / len(numbers)
            
        elif analysis_type == "median":
            sorted_nums = sorted(numbers)
            n = len(sorted_nums)
            if n % 2 == 0:
                return (sorted_nums[n//2 - 1] + sorted_nums[n//2]) / 2
            else:
                return sorted_nums[n//2]
            
        elif analysis_type == "mode":
            return max(set(numbers), key=numbers.count)
            
        elif analysis_type == "range":
            return max(numbers) - min(numbers)
            
        elif analysis_type == "std":
            mean = sum(numbers) / len(numbers)
            variance = sum((x - mean) ** 2 for x in numbers) / len(numbers)
            return variance ** 0.5
            
        else:
            return f"Error: Unknown analysis type '{analysis_type}'"
data_analyzer = DataAnalyzer()


In [ ]:
# VALIDATION TESTS
print("\n🧪 Testing your data_analyzer tool:\n")

def test_data_analyzer():
    test_data = [10, 20, 30, 40, 50]
    
    tests = [
        ("mean", 30.0),
        ("median", 30.0),
        ("range", 40.0),
    ]
    
    for analysis_type, expected in tests:
        try:
            result = data_analyzer.forward(test_data, analysis_type)
            if abs(result - expected) < 0.01:
                print(f"✅ {analysis_type}: {result}")
            else:
                print(f"❌ {analysis_type} failed. Expected {expected}, got {result}")
        except Exception as e:
            print(f"❌ Error testing {analysis_type}: {e}")

# Uncomment to run tests
test_data_analyzer()

print("\n✅ Section 4 complete! Great work building your own tool!\n")


In [ ]:
# ============================================================================
# 🚀 SECTION 5: ADVANCED CHALLENGE - MULTIPLE TOOLS (20 minutes)
# ============================================================================

print("="*80)
print("🚀 SECTION 5: ADVANCED - COMBINING MULTIPLE TOOLS")
print("="*80)

"""
Ready for a challenge? Create an agent that uses MULTIPLE tools together!

EXAMPLE SCENARIO: A "Student Helper" agent with multiple tools:
- Calculator for math
- Text analyzer for essays
- Fun facts for learning

The agent can use ALL tools to help with complex tasks!
"""

print("\n📝 CHALLENGE: Create a multi-tool agent\n")

# TODO: Create 2-3 tools that work well together

# Example: Study Helper Agent

@tool
def study_timer(minutes: int) -> str:
    """
    Calculates study session end time.
    
    Args:
        minutes: Number of minutes to study
    
    Returns:
        A message about the study session
    """
    from datetime import datetime, timedelta
    
    end_time = datetime.now() + timedelta(minutes=minutes)
    return f"Study session of {minutes} minutes will end at {end_time.strftime('%H:%M:%S')}"


@tool
def quiz_generator(topic: str, num_questions: int) -> str:
    """
    Generates quiz questions for studying.
    
    Args:
        topic: The subject to quiz on
        num_questions: How many questions to generate
    
    Returns:
        A set of quiz questions
    """
    questions = {
        "math": [
            "What is the Pythagorean theorem?",
            "Solve: 2x + 5 = 15",
            "What is the derivative of x²?",
        ],
        "science": [
            "What is photosynthesis?",
            "Name the planets in order from the sun",
            "What is Newton's first law?",
        ],
        "history": [
            "When did World War II end?",
            "Who was the first US president?",
            "What year was the Declaration of Independence signed?",
        ]
    }
    
    topic_lower = topic.lower()
    if topic_lower in questions:
        available = questions[topic_lower][:num_questions]
        return "\n".join([f"{i+1}. {q}" for i, q in enumerate(available)])
    else:
        return f"No questions available for topic: {topic}"


# TODO: Create your multi-tool agent


# Create the agent with multiple tools
study_agent = CodeAgent(
    tools=[
        study_timer,
        quiz_generator,
    ],
    model=InferenceClientModel(model_id="Qwen/Qwen2.5-Coder-32B-Instruct"),
    max_steps=10,  # More steps for complex tasks
    verbosity_level=2
)

# Test with a complex, multi-step task
result = study_agent.run('''
    I need to study for my math test. 
    First, calculate how long I have: it's 2:00 PM and my test is at 5:00 PM.
    Then set up a 90-minute study session.
    Finally, generate 3 quiz questions about math to test myself.
''')

print(f"\n✨ Result:\n{result}\n")


print("\n" + "="*80)
print("💡 MULTI-TOOL DESIGN TIPS")
print("="*80)

print("""
When combining tools, consider:

1. 🔗 Tool Synergy: Tools should complement each other
   Example: A web search tool + a summarizer tool

2. 🎯 Clear Boundaries: Each tool should have a distinct purpose
   Don't create overlapping functionality

3. 📊 Data Flow: One tool's output can be another's input
   Example: Image generator → Image analyzer

4. 🛡️ Error Handling: Tools should fail gracefully
   The agent should be able to try alternatives

5. 📝 Documentation: More tools = more important docstrings
   The LLM needs to understand when to use each tool

EXAMPLE MULTI-TOOL SYSTEMS:
- 📰 News Agent: Search + Summarize + Translate
- 🎨 Creative Agent: Image Gen + Text + Music
- 📊 Data Agent: Fetch + Analyze + Visualize
- 🏢 Business Agent: Calculate + Report + Email
""")

print("\n✅ Section 5 complete! You're a multi-tool expert!\n")

In [ ]:
# ============================================================================
# 🚀 SECTION 6: MULTIPLE CHALLENGES 
# ============================================================================

print("="*80)
print("🔗 SEQUENTIAL TOOL CHAINING CHALLENGES")
print("="*80)
print()
print("Concept: These prompts require using multiple tools in sequence, where the")
print("output from one tool becomes the input to the next.")
print()

# SCENARIO 1
print("-"*80)
print("🛒 SCENARIO 1: E-COMMERCE SHOPPING")
print("-"*80)
print()
print("Prompts:")
print()
print("1. 'I want to buy a laptop that costs $1000. There's a 15% discount, then 8%")
print("   tax, and I need international shipping ($50). What's my total cost?'")
print()
print("2. 'A gaming console is $500 with a 20% Black Friday discount. After tax (6%),")
print("   what do I pay?'")
print()
print("3. 'Compare two options: Phone at $800 with 10% discount, 7% tax, domestic")
print("   shipping vs same phone at $750, no discount, 7% tax, domestic shipping.")
print("   Which is cheaper?'")
print()
print("4. 'I'm buying 3 headphones at $150 each, with a bulk discount of 25%, plus 10%")
print("   tax, and express shipping (adds $30). What's the total?'")
print()
print("Tool Hints: Product price lookup, discount calculator, tax calculator,")
print("            shipping calculator")
print()

# SCENARIO 2
print("-"*80)
print("🌍 SCENARIO 2: INTERNATIONAL EVENT PLANNING")
print("-"*80)
print()
print("Prompts:")
print()
print("1. 'I'm in New York and want to schedule a 2-hour meeting at 9 AM my time.")
print("   What time will it be in Tokyo, and when will it end for them?'")
print()
print("2. 'I need to have a 3-hour meeting with people in London, Dubai, and Sydney.")
print("   If I start at 2 PM London time, what time does it end for each location?'")
print()
print("3. 'Find a 1-hour slot that works for Tokyo (9 AM - 6 PM), New York (9 AM - 5 PM),")
print("   and London (9 AM - 6 PM). Start searching from 10 AM Tokyo time.'")
print()
print("4. 'Schedule a 90-minute meeting starting 3 PM Paris time. Check if this works")
print("   for participants in San Francisco and Mumbai (work hours are 9 AM - 6 PM")
print("   in each city).'")
print()
print("Tool Hints: Time zone converter, meeting duration calculator, attendee")
print("            availability checker, calendar slot finder")
print()

# SCENARIO 3
print("-"*80)
print("🏋️ SCENARIO 3: FITNESS & NUTRITION TRACKING")
print("-"*80)
print()
print("Prompts:")
print()
print("1. 'I weigh 70kg and ran for 45 minutes, then ate a burger (800 calories).")
print("   My BMR is 1800. What's my calorie balance?'")
print()
print("2. 'I'm 80kg, did 30min cycling and 20min swimming. I ate pizza (1200 cal)")
print("   and salad (200 cal). My BMR is 2000. Will I lose weight this week if I")
print("   do this daily?'")
print()
print("3. 'I want to lose 2kg in 2 weeks. I weigh 75kg and my BMR is 1900. I can")
print("   run 30min daily. How many calories can I eat per day?'")
print()
print("4. 'I'm 65kg, want to maintain weight. I'll do yoga 45min. I had breakfast")
print("   (400 cal) and lunch (600 cal). BMR is 1700. How many calories can I have")
print("   for dinner?'")
print()
print("Tool Hints: Exercise calorie burner, food calorie lookup, calorie balance")
print("            calculator, weight change predictor")
print()

# SCENARIO 4
print("-"*80)
print("📈 SCENARIO 4: INVESTMENT PORTFOLIO ANALYSIS")
print("-"*80)
print()
print("Prompts:")
print()
print("1. 'I bought 100 shares of Apple at $150 each. Current price is $180. I held")
print("   for over a year. How much profit after 15% long-term capital gains tax?'")
print()
print("2. 'I have a portfolio: 50 shares of Tesla bought at $200 (now $250) and 100")
print("   shares of Microsoft bought at $300 (now $320). What's my total profit and")
print("   which performed better percentage-wise?'")
print()
print("3. 'I want to sell my 200 shares of Google to buy Amazon stock. I bought Google")
print("   at $100, it's now $150. Amazon is currently $120. After short-term capital")
print("   gains tax (30%), how many Amazon shares can I buy?'")
print()
print("4. 'I want 60% stocks, 40% bonds. I have $10,000 in stocks (up 20% from $8,333)")
print("   and $5,000 in bonds (up 5% from $4,762). Rebalance to target allocation.")
print("   How much do I need to move?'")
print()
print("Tool Hints: Stock price fetcher, portfolio value calculator, profit/loss")
print("            calculator, tax on gains calculator")
print()

# SCENARIO 5
print("-"*80)
print("⚡ SCENARIO 5: SMART HOME ENERGY MANAGEMENT")
print("-"*80)
print()
print("Prompts:")
print()
print("1. 'My refrigerator runs 24 hours a day. What does it cost per month and")
print("   what's the carbon footprint? (Electricity is $0.12/kWh)'")
print()
print("2. 'Compare running my AC 8 hours/day vs 4 hours/day for a month. How much")
print("   do I save in cost and carbon? (Rate: $0.15/kWh)'")
print()
print("3. 'I run: TV (6h/day), washing machine (1h/day), dishwasher (1h/day), and")
print("   lights (5h/day). Which appliance costs most? Total monthly cost?")
print("   (Rate: $0.10/kWh)'")
print()
print("4. 'I use 900 kWh/month. Solar panels generate 30 kWh/day. After installing")
print("   solar, what's my new monthly cost and carbon reduction? (Rate: $0.12/kWh)'")
print()
print("Tool Hints: Appliance wattage lookup, energy usage calculator, electricity")
print("            cost calculator, carbon footprint calculator")
print()

# SCENARIO 6
print("-"*80)
print("🍳 SCENARIO 6: RECIPE SCALING & NUTRITION")
print("-"*80)
print()
print("Prompts:")
print()
print("1. 'A recipe for 4 people uses 2 cups flour and 3 eggs. I need to make it for")
print("   6 people. What are the calories per serving? (Flour: 455 cal/cup, Egg:")
print("   70 cal each)'")
print()
print("2. 'Original recipe (serves 2): 200g chicken, 100g rice, 1 tbsp oil. Scale to")
print("   5 servings and calculate total protein. (Chicken: 31g protein/100g, Rice:")
print("   2.7g/100g, Oil: 0g)'")
print()
print("3. 'I'm meal prepping for 7 days. Recipe serves 3, uses: 300g beef (250 cal/100g),")
print("   150g pasta (350 cal/100g), 2 tbsp sauce (50 cal/tbsp). Scale to 7 servings.")
print("   What's the calorie count per meal? Will it fit my 500 cal/meal budget?'")
print()
print("4. 'I'm hosting 12 people. Two recipes: Recipe A (serves 4): 2 cups rice, 400g")
print("   chicken. Recipe B (serves 6): 3 cups lentils, 2 cups vegetables. Calculate")
print("   total calories and cost. (Rice: 200 cal/cup, $2/cup | Chicken: 165 cal/100g,")
print("   $8/kg | Lentils: 230 cal/cup, $3/cup | Vegetables: 50 cal/cup, $1/cup)'")
print()
print("Tool Hints: Recipe scaler, ingredient calories lookup, nutrition calculator,")
print("            macro breakdown")
print()

# SCENARIO 7
print("-"*80)
print("📅 SCENARIO 7: PROJECT TIMELINE & RESOURCE MANAGEMENT")
print("-"*80)
print()
print("Prompts:")
print()
print("1. 'Project starts Jan 1, 2025. Tasks: Design (high complexity), Development")
print("   (high), Testing (medium). Each needs 1 developer at $50/h (8h days). When")
print("   does it finish and what's the cost?'")
print()
print("2. 'Deadline is March 1, 2025. Start date is Feb 1, 2025. Tasks: Planning:")
print("   5 days (2 people, $60/h), Execution: 15 days (3 people, $75/h), Review:")
print("   3 days (1 person, $80/h). Will we make the deadline? What's the total cost?'")
print()
print("3. 'I have $20,000 budget and deadline of April 15 starting April 1. Tasks:")
print("   Research (7 days), Build (12 days), Test (6 days). Try: Option A (1 person")
print("   @ $65/h) vs Option B (2 people @ $40/h). Which fits budget and timeline?'")
print()
print("4. 'Sequential tasks (can't overlap): Requirements: 4 days, Design (depends on")
print("   requirements): 8 days, Development (depends on design): 20 days, Testing")
print("   (depends on development): 7 days. Start: May 1. Team: 2 people @ $55/h.")
print("   What's end date and cost?'")
print()
print("Tool Hints: Task duration estimator, date calculator, resource cost calculator,")
print("            deadline checker")
print()

# KEY INSIGHTS
print("-"*80)
print("💡 KEY INSIGHTS")
print("-"*80)
print()
print("Complexity Levels:")
print("  ⭐ Simple: 2-3 tools in sequence")
print("  ⭐⭐ Moderate: 4-6 tools in sequence")
print("  ⭐⭐⭐ Complex: 7+ tools with branching logic")
print()
print("Bonus Challenge: Create prompts requiring the same tool multiple times in")
print("                 different steps (e.g., 'Convert $100 USD to EUR, then")
print("                 convert that EUR to GBP')")
print()
print("="*80)